In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.utils.functions failed: Traceback (most recent call last):
  File "c:\Users\jerem\anaconda3\envs\master\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\jerem\anaconda3\envs\master\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\jerem\anaconda3\envs\master\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\jerem\Documents\Informatique\AdaCourse\ada-2024-project-lesfraisestagada\src\utils\functions.py", line 9, in <module>
    from src.scripts.articles_clicks_links import bad_articles
ImportError: cannot import name 'bad_articl

In [11]:
from src.scripts.article_to_country import *
from src.data.dataloader import *
from src.utils.plots import *
from src.utils.functions import *
from itertools import combinations
import plotly.express as px


# Country classification Analysis

</div><div class="alert alert-success"> <strong>First glance at the distribution of knowledge worldwide :</strong> 

- Assign a country to each article
- Try different methods and see how many articles can be classified
- Verify these classification with human annotation 
- Select the best method for further analysis of the dataset
</div>

</div> <div class="alert alert-warning">  <strong>Countries :</strong>

In order to classify countries we used the standar ISO list with 249 countries and the following distribution: 
- UN Members: 193
- UN Observer States: 2
- States With Partial Recognition: 2
- Inhabited Dependent Territories: 45
- Uninhabited Territories: 6
- Antarctica: 1
</div>

A naïve method was employed to classify articles to specific countries by performing a text search to identify occurrences of country names within the plaintext. This approach utilized basic regular expression matching to analyze the articles. However, it resulted in approximately 31% of the articles (1,412 out of 4,604) remaining unclassified, highlighting significant limitations.

### Identified Issues with the Approach:
1. **Over-classification on mentioned country names**: 
   The presence of a country name in an article does not necessarily imply that the article belongs to that country. This assumption leads to inaccuracies in classification.

2. **Lack of contextual understanding**: 
   The text search method lacks the ability to discern the context in which a country name appears, resulting in potential misclassifications.

3. **Observations**: 
   A review of the articles revealed a lot of incorrect or missing classifications. For example:
   - The article *13th Century* was missclassified under China due to mentions of events occurring there during that period, despite the article lacking any specific association with China.
   - Conversely, the article *4-2-0*, which details a type of railroad in the United States, was left unclassified despite its clear appartenance to that country.

These findings show the limitations of the current text-matching methodology and highlight the need for a more robust, context-aware approach to accurately classify articles.

Given the limitations of the initial text-search-based approach, a decision was made to use Large Language Models (LLMs) to enhance the article classification process. LLMs, with their extensive general knowledge, have the capability to analyze articles and respond to queries with a deeper understanding. 

Unlike simple text searches, LLMs not only utilize the content of the article but also incorporate their pre-existing (trained) knowledge. This contextual understanding should enables more accurate classification.

By employing LLMs, two potential strategies can be pursued:
1. **Targeted classification of previously unclassified articles**: The 1,412 articles that remained unclassified under the initial approach can be reevaluated, allowing for a significant reduction in the unclassified proportion. After this step only 6% (283 out of 4604) were not classified.

2. **Full Reclassification**: LLMs can be used to perform a complete reclassification of all articles, eliminating the biases and limitations inherent in the original text-search methodology. In order to test this two existing LLMs were used : [LLaMa](https://www.llama.com/) and [Qwen](https://qwen-ai.com/). These powerful models, with less than 8 billion parameters, offer significant computational efficiency and can be run locally on consumer hardware, making them accessible and cost-effective solutions compared to bigger models like ChatGPT and their paid API.

This transition to LLM-based classification is expected to significantly improve the accuracy and reliability of the country assignment process.

The prompt used in order to make the models classify the articles was the following : 

```
You will be given textual articles. For each article provide a single and unique country to which the article is related and should be classified to. Provide the answer in the form : country.
If there is no country related to the article, please write 'None'. 
If the location is not on earth, please write 'None'. 
If the article is a general article where the content is not specifically related to a country, please write 'None'.
You must be 100\% sure this is a question of life.
This is the list of coutnries that you are allowed to output don't output anything that is not in this list: {countries}
```

To test the accuracy of the model’s predictions compared to human judgment, each member of the group manually annotated 20 articles, with a 10-article overlap between annotators. As a result, each article was annotated by two members, yielding a total of 50 annotations. Among these, 36 annotations matched, resulting in an inter-annotator agreement of 72%. This annotated subset was used as a benchmark to evaluate various classification methods and establish an agreement metric.

The highest agreement with human annotations (72%) was achieved using the "Full Classification with LLaMa." However, a review of the assignments revealed that an excessively high number of articles (90%) were being classified, leading to potential overclassification. Some articles were misclassified because the system prompt provided the model with a list of all countries. This introduced a bias, leading the model to disproportionately classify articles under Afghanistan, as it appears first alphabetically in the list.

To address this, the system prompt was iteratively refined to enhance agreement accuracy. After achieving improved agreement values, the refined prompt was used to reclassify the entire dataset. In order to remove the biases due to the ordering of countries in the list the classification was run 2 times with two different orders and then only the matching assignments were kept.

The improved prompt is : 

```
You will be given textual articles. For each article provide a single and unique country to which the article is related and should be classified to. Provide the answer in the form : country. 
If the article is related to an object, a place, a monument related to a country, please write the country.
if the article is about a spieces, that lives in multiple countries, please write 'None'.
If there is no country related to the article, please write 'None'. 
If the location is not on earth, please write 'None'. 
If the article is a general article where the content is not specifically related to a country, please write 'None'.
You are allowed to use the article name to help you find the country.
This is the list of coutnries that you are allowed to output don't output anything that is not in this list: {countries}
```

This refinement resulted in an improved agreement value of 78%% while reducing the proportion of classified articles to 56%, addressing the issue of overclassification. This proportion of classified articles matches our expectations and is possibly still a bit high since in our annotation 41% of articles were not classified.
This final classification is then used for the whole project.

### Downsides and limitations : 
- **Limited number of annotated articles** : The analysis was based on a relatively small sample of 36 annotated articles, which may result in imprecise agreement values. To improve the accuracy and reliability of the findings, a larger dataset of annotated articles would be necessary. However, due to time constraints, expanding the dataset or engaging additional human annotators was not feasible.
- **Small size of LLM** : The model used in this study was relatively small, which inherently limits its knowledge and performance. While larger language models are expected to perform better on such tasks, the decision to prioritize local execution and cost-effectiveness constrained the use of more powerful models.

### Below you will find the code showing how the annotation was done and how the plots were generated

For more detailed informations about how the classifications were done please refer to the script ```article_to_country.py```

In [12]:
results = {}
country_data = pd.read_csv('data/country_data.csv', index_col=0)
counts = filter_top_k(country_data, k=2, N=1)
total_number_of_articles = len(counts)

nan_df = counts[counts.isna().all(axis=1)]
print(f"Number of articles with no countries before completion with llama: {len(nan_df)}")
results["Text search"] = (total_number_of_articles - len(nan_df)) / total_number_of_articles * 100

refined_data = pd.read_csv("data/country_occurences.csv", index_col=0)
nan_df = refined_data[refined_data.isna().all(axis=1)]
print(f"Number of articles with no countries after completion with naive + llama: {len(nan_df)}")
results["Text search + missing articles classified with LlaMa"] = (total_number_of_articles - len(nan_df)) / total_number_of_articles * 100

qwen_country_data = pd.read_csv('data/country_data_full_qwen.csv', index_col=0)
qwen_missing = len(qwen_country_data[qwen_country_data["Top_1_name"].isna()])
print(f"Number of articles with no countries after completion with Qwen: {qwen_missing}")
results["Full classification with Qwen"] = (total_number_of_articles - qwen_missing) / total_number_of_articles * 100


llama_country_data = pd.read_csv('data/country_data_full_llama.csv', index_col=0)
llama_missing = len(llama_country_data[llama_country_data["Top_1_name"].isna()])
print(f"Number of articles with no countries after completion with LLAMA: {llama_missing}")
results["Full classification with LLaMa"] = (total_number_of_articles - llama_missing) / total_number_of_articles * 100

llama_country_data_imporved_normal = pd.read_csv('data/country_data_full_llama_improved_normal.csv', index_col=0)
llama_country_data_imporved_reversed = pd.read_csv('data/country_data_full_llama_improved_reversed.csv', index_col=0)

llama_country_data_imporved = pd.read_csv('data/country_data_full_llama_improved_reversed.csv', index_col=0)

llama_country_data_imporved_normal["Top_1_name"] = llama_country_data_imporved_normal["Top_1_name"].where(llama_country_data_imporved_normal['Top_1_name'] == llama_country_data_imporved_reversed['Top_1_name'], np.nan)
llama_country_data_imporved = llama_country_data_imporved_normal
llama_missing_improved = len(llama_country_data_imporved[llama_country_data_imporved["Top_1_name"].isna()])
print(f"Number of articles with no countries after completion with LLAMA: {llama_missing_improved}")
results["Improved classification with LLaMa"] = (total_number_of_articles - llama_missing_improved) / total_number_of_articles * 100
llama_country_data_imporved.drop(columns=["Predictions"], inplace=True)
llama_country_data_imporved.to_csv('data/country_data_full_llama_improved.csv')


Number of articles with no countries before completion with llama: 1412
Number of articles with no countries after completion with naive + llama: 283
Number of articles with no countries after completion with Qwen: 2905
Number of articles with no countries after completion with LLAMA: 479
Number of articles with no countries after completion with LLAMA: 2010


In [13]:
fig = px.bar(x=results.keys(), y=results.values(), color=results.keys(), labels={"x": "Classification Method", "y": "Proportion of articles assigned to a country in %"})
fig.update_layout(title="Proportion of articles assigned to a country")
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_html("./graphs/proportion_country_assignment.html")

In [15]:
models = {
    "Text search": counts,
    "Text search + missing articles classified with LlaMa": refined_data,
    "Full classification with Qwen": qwen_country_data,
    "Full classification with LLaMa": llama_country_data,
    "Improved classification with LLaMa": llama_country_data_imporved
}

heatmap_data = pd.DataFrame(index=models.keys(), columns=models.keys())

for model in models.keys():
    heatmap_data.loc[model, model] = 1.0

for model1, model2 in combinations(models.keys(), 2):
    df_1 = models[model1]
    df_2 = models[model2]
    overlap = (df_1["Top_1_name"] == df_2["Top_1_name"]).sum() / total_number_of_articles
    heatmap_data.loc[model2, model1] = overlap.round(2)

heatmap_data = heatmap_data.astype(float)

heatmap_long = heatmap_data.reset_index().melt(id_vars="index", var_name="Classification Method 2", value_name="Overlap")
heatmap_long.rename(columns={"index": "Classification Method 1"}, inplace=True)

fig = px.imshow(
    heatmap_data.values,
    labels={"x": "Classification Method 2", "y": "Classification Method 1", "color": "Overlap"},
    x=heatmap_data.columns,
    y=heatmap_data.index,
    color_continuous_scale="Blues",
    text_auto=True,
)
fig.update_layout(title="Overlap Heatmap of classification methods", xaxis_title="Classification Method 2", yaxis_title="Classification Method 1")
fig.update_layout(
    xaxis_tickangle=35,
    # yaxis_tickangle=45
)
fig.show()
fig.write_html("./graphs/overlap_heatmap.html")

In [6]:
articles = pd.DataFrame(counts.index)

subset_1 = articles.sample(10, random_state=0).values.flatten().tolist()
subset_2 = articles.sample(10, random_state=1).values.flatten().tolist()
subset_3 = articles.sample(10, random_state=2).values.flatten().tolist()
subset_4 = articles.sample(10, random_state=3).values.flatten().tolist()
subset_5 = articles.sample(10, random_state=4).values.flatten().tolist()
subset_6 = articles.sample(10, random_state=5).values.flatten().tolist()

claire = subset_1 + subset_3
theo = subset_2 + subset_4
oriane = subset_1 + subset_4
bryan = subset_2 + subset_5
jeremy = subset_3 + subset_5

claire = pd.DataFrame(index=claire, columns=["country"])
theo = pd.DataFrame(index=theo, columns=["country"])
oriane = pd.DataFrame(index=oriane, columns=["country"])
bryan= pd.DataFrame(index=bryan, columns=["country"])
jeremy = pd.DataFrame(index=jeremy, columns=["country"])

# pd.DataFrame(claire).to_csv("claire.csv")
# pd.DataFrame(theo).to_csv("theo.csv")
# pd.DataFrame(oriane).to_csv("oriane.csv")
# pd.DataFrame(bryan).to_csv("bryan.csv")
# pd.DataFrame(jeremy).to_csv("jeremy.csv")

# countries = list(dict(countries_for_language('en')).values())
# pd.DataFrame(countries).to_csv("countries.csv")

annotation_path = "./data/annotated/"

claire = pd.read_csv(annotation_path + "subset_claire.csv", index_col=0, na_values="None")
theo = pd.read_csv(annotation_path + "subset_theo.csv", index_col=0, na_values="None")
oriane = pd.read_csv(annotation_path + "subset_oriane.csv", index_col=0, na_values="None")
bryan = pd.read_csv(annotation_path + "subset_bryan.csv", index_col=0, na_values="None")
jeremy = pd.read_csv(annotation_path + "subset_jeremy.csv", index_col=0, na_values="None")

subset_1_c = claire[:10]
subset_3_c = claire[10:]

subset_2_t = theo[:10]
subset_4_t = theo[10:]

subset_1_o = oriane[:10]
subset_4_o = oriane[10:]

subset_2_b = bryan[:10]
subset_5_b = bryan[10:]

subset_3_j = jeremy[:10]
subset_5_j = jeremy[10:]

comparison1 = subset_1_c["country"].str.lower().fillna("nan") == subset_1_o["country"].str.lower().fillna("nan")
print(f"Agreement between Claire and Oriane: {comparison1.sum() * 10}%")
comparison2 = subset_2_t["country"].str.lower().fillna("nan") == subset_2_b["country"].str.lower().fillna("nan")
print(f"Agreement between Theo and Bryan: {comparison2.sum() * 10}%")
comparison3 = subset_3_c["country"].str.lower().fillna("nan") == subset_3_j["country"].str.lower().fillna("nan")
print(f"Agreement between Claire and Jeremy: {comparison3.sum() * 10}%")
comparison4 = subset_4_t["country"].str.lower().fillna("nan") == subset_4_o["country"].str.lower().fillna("nan")
print(f"Agreement between Theo and Oriane: {comparison4.sum() * 10}%")
comparison5 = subset_5_b["country"].str.lower().fillna("nan") == subset_5_j["country"].str.lower().fillna("nan")
print(f"Agreement between Bryan and Jeremy: {comparison5.sum() * 10}%")

df = pd.concat([
    subset_1_c.loc[comparison1],
    subset_2_b.loc[comparison2],
    subset_3_c.loc[comparison3],
    subset_4_t.loc[comparison4],
    subset_5_b.loc[comparison5]
], ignore_index=False)

df.to_csv("data/annotated/consensus.csv")
llama_country_data_imporved.loc[df.index]["Top_1_name"].str.lower().fillna("nan")

Agreement between Claire and Oriane: 80%
Agreement between Theo and Bryan: 60%
Agreement between Claire and Jeremy: 80%
Agreement between Theo and Oriane: 50%
Agreement between Bryan and Jeremy: 90%


Hadrian%27s_Wall                                    nan
Globe_Theatre                                       nan
List_of_monarchs_in_the_British_Isles    united kingdom
Aleksandr_Vasilevsky                             russia
Pyongyang                                   north korea
Ivory                                               nan
Beep_%28SWBLW%29                          united states
Fermium                                             nan
Dwight_D._Eisenhower                                nan
Invasion                                        grenada
Scooby-Doo                                united states
Bill_Clinton                              united states
Frederick_Douglass                        united states
Cambrian_explosion                                china
CPU_cache                                           nan
Cathedral_of_Magdeburg                          germany
Anna_Karenina                                    russia
Eurasian_Bullfinch                              

In [7]:
models = {
    "Text search": counts,
    "Text search + missing articles classified with LlaMa": refined_data,
    "Full classification with Qwen": qwen_country_data,
    "Full classification with LLaMa": llama_country_data,
    "Improved classification with LLaMa": llama_country_data_imporved
}

values = []
for model in models.keys():
    agreement_value = (models[model].loc[df.index]["Top_1_name"].str.lower().fillna("nan") == df["country"].str.lower().fillna("nan")).sum() / len(df) * 100
    values.append(agreement_value)
    
fig = px.bar(
    x=models.keys(), 
    y=values, 
    color=models.keys(),
    title="Agreement between annotators and classification method in %", 
    labels={"x": "Classification Method", "y": "Agreement value in %"}
    )

fig.update_yaxes(range=[0, 100])
fig.show()

fig.write_html("./graphs/agreement_bar_plot.html")